In [2]:
import torch
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import math
# torch.set_printoptions({'float': '{: 0.8f}'.format})
torch.set_printoptions(sci_mode=False, precision=8)


Load df with y_col

In [18]:
df = pd.read_csv("./footballData/CombinedSlidingWindow4.csv", index_col=False, low_memory=False)
y_col = ['H_Won', 'H_start_odds', 'V_start_odds']
y_df = df[y_col]

# Custom loss function
**Input:** X and Y where each value is between -1 and 1. Both corresponding to the probability of home (+) vs away (-) where 0.5 = 50%, 1.0=100% home chance of winning, -0.7 = 70% visitor chance of winning etc.

Also the odds will be in the input as well

We will use Adam as the optimizer without passing in any. So by default smaller values are

1. Calculate the kelly criterion based on the predicted probability. When it suggests a negative number (no bet) set both X and y to 0
2. Calculate the pearson correlation coefficient, get its absolute value and multiply it by a hyperparameter constant. Combine this with the 



In [52]:
"""
Want: 
    An array > 1 elements for PCC formula
    A smaller array for kelly_criterion
    


x: 1d array of predictions between -1 and 1 where negative number means visitor predicted to win
y: ['H_Won', 'H_start_odds', 'V_start_odds']
pearson_multiplier: constant to multiply the pearson correlation coefficient's result by
max_bet_size: Amount to multiply to kelly criterion
"""
def nfl_custom_criterion(x, y, pearson_multiplier=0.5, max_bet_size=100):
    # ------------------------------------------------
    # Preliminary calculations
    # ------------------------------------------------
    h_start_odds = y[:,1]
    v_start_odds = y[:,2]
    h_won = y[:,0]
    y_decimal_odds = torch.where(x > 0, h_start_odds, v_start_odds) # Predicted vs actual odds (regardless of correct prediction)
    y_prob = 1 / y_decimal_odds                  # Probability (regardless of correct prediction)
    x_H_Won = torch.round(torch.sigmoid(20 * x)) # Sigmoid so that it's differentiable. The 20 is arbitrarily large number
    y_correct_prediction = torch.abs((x_H_Won - h_won))        # 1 if wrong bet, otherwise 0. Used to reset kelly when wrong
    y_correct_prediction_mult_two = 2 * y_correct_prediction   # 2 if wrong bet, 0 if correct
    x = torch.abs(x)

    
    # ------------------------------------------------
    # 1. Calculate the Pearson Correlation Coefficient
    # ------------------------------------------------
    n = x.size(0)
    sum_x = torch.sum(x)
    sum_x_squared = torch.sum(x**2)
    sum_y = torch.sum(y_prob)
    sum_y_squared = torch.sum(y_prob**2)
    sum_pow_x = torch.sum(x**2)
    sum_pow_y = torch.sum(y_prob**2)
    x_mul_y = torch.mul(x, y_prob)
    sum_x_mul_y = torch.sum(x_mul_y)

    
    # PCC Formula (eps to avoid NaN)
    eps = 1e-8
    pcc_numerator = n * sum_x_mul_y - sum_x * sum_y
    pcc_denominator_one = torch.sqrt(n * sum_pow_x - sum_x_squared + eps)
    pcc_denominator_two = torch.sqrt(n * sum_pow_y - sum_y_squared + eps)
    pcc = pcc_numerator / (pcc_denominator_one * pcc_denominator_two + eps)
    pcc = pearson_multiplier * torch.abs(pcc)

    
    # ------------------------------------------------
    # 2. Calculate the kelly criterion
    #    Entirely wrong predictions are negated and kept in "incorrect_bets" (pcc not applied to wrong predictions)
    #    Correct predictions are kept in "correct_bets". Pcc is applied to this & stored in pcc_adjusted_correct_bets
    #    Possible issue: This always bets max_bet_size
    #    The result is cumulatively calculated. i.e. The sum of the previous values are used to calculate the next one
    # ------------------------------------------------
    kelly_criterion = x - ((1 - x) / y_decimal_odds)
    bet_multiplier = torch.clamp(kelly_criterion, min=0)   # Kelly results that are negative are ignored
    bet_unadjusted_profit = bet_multiplier*max_bet_size    # Assumes all bets were correct

    correct_bets = bet_unadjusted_profit - (bet_unadjusted_profit * y_correct_prediction)   # All correct bets after kelly, profit or 0.
    pcc_adjusted_correct_bets = correct_bets * (1 - pcc)                                    # "correct_bets" penalized by pcc
    incorrect_bets = bet_unadjusted_profit - (bet_unadjusted_profit * y_correct_prediction_mult_two) # Negative numbers are incorrect bets
    incorrect_bets = torch.clamp(incorrect_bets, max=0)  # Restrict to 0 or negative.
    combined_bets = correct_bets + incorrect_bets        # Profit


    # ------------------------------------------------
    # Combine & Return
    #     Negate everything for Adam & optuna
    # ------------------------------------------------
    return -torch.cumsum(combined_bets, dim=0)[-1]

In [51]:
x = torch.rand(y_df.shape[0]) * 2 - 1
print(x)
y = torch.tensor(y_df.values, dtype=torch.float32)
n_splits = math.ceil(y_df.shape[0] / 6)

# Split intp batches
BATCH_SIZE = 10
dataset = TensorDataset(x, y)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)

for id_batch, (x_batch, y_batch) in enumerate(dataloader):
    # print(x_batch.shape)
    # print(y_batch)
    res_df = nfl_custom_criterion(x_batch, y_batch)
    print(res_df)
    
#print(y)
# print(y[:,1])
# res_df = nfl_custom_criterion(x,y)
#print(x)
# print(res_df)
# for item in res_df:
#    print(item)

tensor([-0.78928792,  0.87805712, -0.01085651,  ..., -0.80255032,
        -0.49695253,  0.14454949])
7.399600982666016
179.8631591796875
-272.7281188964844
-1.3484268188476562
-19.84394073486328
51.71514892578125
0.10350227355957031
11.50827407836914
4.187017440795898
-23.521915435791016
-149.28488159179688
-173.93966674804688
-136.4744415283203
246.40625
217.01480102539062
-40.612037658691406
171.76333618164062
-86.79293823242188
318.6509704589844
-38.51661682128906
-88.43992614746094
-129.51039123535156
84.99119567871094
-44.08207702636719
-62.958736419677734
-155.43887329101562
-39.068878173828125
120.21916961669922
-53.61178207397461
17.371307373046875
213.08981323242188
8.318538665771484
-145.79495239257812
8.842787742614746
58.496253967285156
-233.984375
11.674960136413574
-119.57804107666016
159.61607360839844
174.83535766601562
-16.757221221923828
38.47792053222656
171.56666564941406
147.903564453125
20.201093673706055
80.85928344726562
23.511857986450195
-25.369468688964844
-1

In [3]:
test_tensor = torch.tensor([100.0000,   0.9000,   1.0981,   0.9000,   0.9353,   1.0000,   1.0981,
                              1.0981,   1.0758,   1.0000,   0.9000,   1.0825,   1.0981,   0.9332,
                              0.9358,   0.9000,   1.0000,   1.0064,   1.0981,   1.0179,   1.0981,
                              1.0981,   1.0639,   1.0840,   0.9000,   1.0981,   1.0981,   1.0934,
                              1.0782,   1.0000,   1.0981,   1.0448,   1.0981,   1.0981,   0.9000,
                              0.9470,   1.0500,   0.9000,   0.9225,   0.9779,   1.0981,   1.0000,
                              1.0977,   1.0014,   1.0832,   1.0000,   1.0000,   1.0670,   1.0319,
                              1.0981,   1.0981,   1.0981,   1.0981,   1.0981,   1.0981,   1.0981,
                              0.9777,   1.0883,   1.0981,   1.0981,   0.9969,   0.9000,   1.0000,
                              1.0237,   1.0094,   1.0665,   1.0569,   0.9310,   1.0656,   0.9110,
                              1.0981,   1.0981,   1.0981,   1.0412,   1.0000,   1.0963,   0.9614,
                              0.9186,   1.0252,   1.0604,   0.9000,   1.0237,   1.0000,   0.9000,
                              1.0981,   0.9000,   1.0981,   1.0000,   1.0157,   1.0807,   1.0408,
                              1.0692,   0.9000,   1.0000,   1.0823,   1.0454,   1.0000,   1.0870,
                              1.0981,   1.0981,   1.0000,   1.0000,   1.0000,   1.0000,   1.0981,
                              1.0981,   1.0000,   0.9616,   1.0000,   1.0000,   1.0981,   1.0981,
                              1.0981,   1.0981,   0.9741,   1.0478,   1.0735,   1.0981,   0.9563,
                              1.0355,   1.0981,   1.0981,   1.0246,   1.0000,   1.0981,   1.0000,
                              0.9377,   1.0000,   0.9736,   1.0449,   0.9000,   1.0000,   1.0770,
                              1.0000,   1.0981,   1.0000,   1.0837,   1.0000,   1.0981,   0.9294,
                              1.0000,   1.0551,   1.0677,   1.0782,   1.0000,   1.0981,   1.0981,
                              1.0652,   1.0981,   1.0432,   0.9995,   1.0981,   1.0000,   1.0981,
                              1.0843,   0.9718,   1.0000,   1.0336,   0.9945,   1.0000,   1.0000,
                              1.0981,   0.9000,   0.9837,   1.0000,   1.0981,   1.0981,   1.0757,
                              1.0981,   1.0014,   1.0672,   0.9000,   1.0981,   1.0981,   0.9000,
                              0.9000,   1.0981,   1.0000,   1.0981,   1.0288,   0.9684,   1.0981,
                              0.9300,   0.9952,   1.0981,   1.0000,   0.9000,   1.0000,   1.0981,
                              0.9000,   1.0981,   0.9775], dtype=torch.float32)

In [5]:
print(torch.cumprod(test_tensor, dim=0))

tensor([  100.00000000,    90.00000000,    98.82899475,    88.94609070,
           83.19127655,    83.19127655,    91.35234070,   100.31399536,
          107.91779327,   107.91779327,    97.12601471,   105.13890839,
          115.45302582,   107.74076080,   100.82380676,    90.74142456,
           90.74142456,    91.32216644,   100.28086853,   102.07589722,
          112.08953857,   123.08551025,   130.95068359,   141.95053101,
          127.75547791,   140.28828430,   154.05055237,   168.43887329,
          181.61079407,   181.61079407,   199.42680359,   208.36112976,
          228.80134583,   251.24674988,   226.12207031,   214.13760376,
          224.84448242,   202.36001587,   186.67712402,   182.55155945,
          200.45986938,   200.45986938,   220.04478455,   220.35285950,
          238.68620300,   238.68620300,   238.68620300,   254.67819214,
          262.80242920,   288.58334351,   316.89334106,   347.98056030,
          382.11743164,   419.60314941,   460.76620483,   505.96

In [6]:
# Generate a tensor with random numbers between -1 and 1
amt_random_numbers = 10
x = torch.rand(amt_random_numbers) * 2 - 1
y = torch.rand(amt_random_numbers) * 2 - 1
# print(type(y))
# print(y.shape)
print(x)
print(y)

nfl_test(x,y)

tensor([-0.38738823, -0.00858855, -0.93326747, -0.30262041, -0.12851167,
         0.99726415, -0.58714378,  0.01136839,  0.43507338,  0.94447637])
tensor([ 0.08781946, -0.13150311,  0.77710104,  0.49798858, -0.76413476,
        -0.15960681, -0.77662611, -0.44353426, -0.09408712, -0.46206462])


NameError: name 'nfl_test' is not defined